In [1]:
import mvpa2
from os.path import exists, join  
import mvpa2.suite as mv
import glob
import numpy as np

/dartfs-hpc/rc/home/4/f002d44/.local/lib/python2.7/site-packages/mvpa2/testing/tools.py:81: DeprecationWarning: Importing from numpy.testing.decorators is deprecated since numpy 1.15.0, import from numpy.testing instead.
  from numpy.testing.decorators import skipif


In [2]:
# set everything up ! 
subid = [1,12,17,27,32,33,34,36,37,41]
subjs = ['{:0>6}'.format(i) for i in subid]
hemis = ['lh', 'rh']
data_path = '/dartfs-hpc/scratch/psyc164/mvpaces/glm/'
datalad_path = '/dartfs/rc/lab/D/DBIC/DBIC/datasets.datalad.org/labs/haxby/attention/derivatives/'


# change accordingly
tas ks = ['beh','tax']
sub, hemi = subjs[0], hemis[0]
prefix = 'sub-rid'+sub
suffix = hemi+'.coefs.gii'


taxonomy = np.repeat(['bird', 'insect', 'primate', 'reptile', 'unduate'],4)
behavior = np.tile(['eating', 'fighting', 'running', 'swimming'],5)
conditions = [' '.join((beh, tax)) for beh, tax in zip(behavior, taxonomy)]

In [3]:
# grab all subj1's  files into a list
#fn = data_path+sub_pref+'*lh.coefs.gii'
fn = data_path+prefix+'*'+suffix
files = sorted(glob.glob(fn))

In [5]:
targets = range(1,21)
ds = None
for x in range(len(files)):
    chunks = [x+1]*20
    d = mv.gifti_dataset(files[x], chunks=chunks, targets=targets)
    d.sa['conditions']=conditions
    d.sa['taxonomy'] = taxonomy
    d.sa['behavior'] = behavior
    if ds is None:
        ds = d
    else:      
        ds = mv.vstack((ds,d))

/dartfs-hpc/rc/home/4/f002d44/.local/lib/python2.7/site-packages/mvpa2/datasets/gifti.py:77: DeprecationWarning: giftiio.read function deprecated. Use nibabel.load() instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  samples = giftiio.read(samples)


In [6]:
ds.fa['node_indices']=range(ds.shape[1])
train='behavior'

In [7]:
# load surface and get searchlight query
radius = 3 
surface = mv.surf.read(join(data_path, '{0}.pial.gii'.format(hemi)))
# this is an arbitrary radius and distance metric!
query = mv.SurfaceQueryEngine(surface, radius, distance_metric='dijkstra')
# based off PyMVPA tutorial
clf = mv.LinearCSVMC()
cv = mv.CrossValidation(clf, mv.NFoldPartitioner(attr=train),
                     errorfx=lambda p, t: np.mean(p == t),
                     enable_ca=['stats'])
searchlights = mv.Searchlight(cv, queryengine=query, postproc=mv.mean_sample(), roi_ids=None)

/dartfs-hpc/rc/home/4/f002d44/.local/lib/python2.7/site-packages/mvpa2/support/nibabel/surf_gifti.py:51: DeprecationWarning: giftiio.read function deprecated. Use nibabel.load() instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  g = giftiio.read(fn)
/dartfs-hpc/rc/home/4/f002d44/.local/lib/python2.7/site-packages/mvpa2/support/nibabel/surf_gifti.py:25: DeprecationWarning: getArraysFromIntent method deprecated. Use get_arrays_from_intent instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  ar = g.getArraysFromIntent(intent)


In [ ]:
sl_clf_results = searchlights(ds)

/dartfs-hpc/rc/home/4/f002d44/.local/lib/python2.7/site-packages/mvpa2/misc/attrmap.py:153: FutureWarning: Conversion of the second argument of issubdtype from `str` to `str` is deprecated. In future, it will be treated as `np.string_ == np.dtype(str).type`.
  if not np.issubdtype(attr.dtype, str) and not self.mapnumeric:
/dartfs-hpc/rc/home/4/f002d44/.local/lib/python2.7/site-packages/mvpa2/clfs/libsvmc/svm.py:215: FutureWarning: Conversion of the second argument of issubdtype from `'c'` to `str` is deprecated. In future, it will be treated as `np.string_ == np.dtype('c').type`.
  if ( np.issubdtype(self.ca.trained_targets.dtype, 'c') or
/dartfs-hpc/rc/home/4/f002d44/.local/lib/python2.7/site-packages/mvpa2/clfs/libsvmc/svm.py:216: FutureWarning: Conversion of the second argument of issubdtype from `'U'` to `unicode` is deprecated. In future, it will be treated as `np.unicode_ == np.dtype('U').type`.
  np.issubdtype(self.ca.trained_targets.dtype, 'U') ):


 * Please note: warnings are printed only once, but underlying problem might occur many times *
